In [3]:
from bs4 import BeautifulSoup
# from lxml import etree
import pandas as pd
import requests
import re
from tqdm import tqdm
import pickle

In [223]:
site_url = 'https://en.wikipedia.org'
base_url = 'https://en.wikipedia.org/wiki/Category:'

In [248]:
def get_song_pages(year, year_subpage_url):
    song_pages = {}

    url = year_subpage_url

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    song_columns = soup.find_all('div', {'class': 'mw-category-group'})

    for c in song_columns:
        try:
            if c.find('h3').text >= list(song_pages)[-1][0]: # minimize redundant retrieval
                song_urls = c.find_all('a', href=True)
                for u in song_urls:
                    song_pages[u.text] = (f"{site_url}{u['href']}", year)

        except:
            song_urls = c.find_all('a', href=True)
            for u in song_urls:
                song_pages[u.text] = (f"{site_url}{u['href']}", year)

    return song_pages

In [249]:
def get_year_subpages(year_page_url):
    year_subpages = {}
    url = year_page_url

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    toc = soup.find('div', {'id': 'toc'})

    try:
        subpage_div = toc.find_all('li')[-1]
    except:
        subpage_div = toc.find('div', {'class': 'large-cat-toc-body'})

    subpage_urls = subpage_div.find_all('a', href=True)
    for u in subpage_urls:
        year_subpages[u.text] = u['href']

    return year_subpages

In [250]:
def get_year_pages(first_decade, last_decade):
    pattern = re.compile('\d{4} songs')

    year_pages = {}

    for decade in range(first_decade, last_decade + 10, 10):
        endpoint = f'{decade}s_songs'
        page = requests.get(f'{base_url}{endpoint}')
        soup = BeautifulSoup(page.content, 'html.parser')
        categories = soup.find_all('div', {'class': 'mw-category mw-category-columns'})
        for c in categories:
            urls = c.find_all('a', href=True)
            for u in urls:
                if bool(pattern.match(u.text)):
                    year_pages[u.text.split()[0]] = f"{site_url}{u['href']}"

    return year_pages

In [254]:
songs_for_year = {}
songs_by_year = []

years = get_year_pages(1950,2010)
for year, y_url in tqdm(years.items()): 
    print('Year:',year)
    subpages = get_year_subpages(y_url)
    for subpage, s_url in tqdm(subpages.items()):
        songs = get_song_pages(year, s_url)
        songs_for_year = songs_for_year | songs
    
    song_list = [(k, v[0], v[1]) for k,v in songs_for_year.items()]
    songs_by_year.extend(song_list)

    print(f'Songs Collected: {len(songs_by_year)}')
    
with open('wiki_songs.pickle', 'wb') as handle:
    pickle.dump(songs_by_year, handle)
    
songs_by_year

  0%|          | 0/70 [00:00<?, ?it/s]

Year: 1950


  1%|▏         | 1/70 [00:05<05:51,  5.09s/it]

Songs Collected: 135
Year: 1951


  3%|▎         | 2/70 [00:10<06:10,  5.44s/it]

...

 99%|█████████▊| 69/70 [27:09<00:46, 46.91s/it]

Songs Collected: 1948506
Year: 2019


100%|██████████| 70/70 [28:04<00:00, 24.06s/it]


Songs Collected: 2038724


[('Accidents Will Happen (Bing Crosby song)',
  'https://en.wikipedia.org/wiki/Accidents_Will_Happen_(Bing_Crosby_song)',
  '1950'),
 ("Adelaide's Lament",
  'https://en.wikipedia.org/wiki/Adelaide%27s_Lament',
  '1950'),
 ('Adoro a mi tierra',
  'https://en.wikipedia.org/wiki/Adoro_a_mi_tierra',
  '1950'),
 ('(Ah, the Apple Trees) When the World Was Young',
  'https://en.wikipedia.org/wiki/(Ah,_the_Apple_Trees)_When_the_World_Was_Young',
  '1950'),
 ('All My Love (Patti Page song)',
  'https://en.wikipedia.org/wiki/All_My_Love_(Patti_Page_song)',
  '1950'),
 ('American Beauty Rose (song)',
  'https://en.wikipedia.org/wiki/American_Beauty_Rose_(song)',
  '1950'),
 ('Anema e core (song)',
  'https://en.wikipedia.org/wiki/Anema_e_core_(song)',
  '1950'),
 ('Anytime, Any Place, Anywhere',
  'https://en.wikipedia.org/wiki/Anytime,_Any_Place,_Anywhere',
  '1950'),
 ("Auf Wiederseh'n, Sweetheart",
  'https://en.wikipedia.org/wiki/Auf_Wiederseh%27n,_Sweetheart',
  '1950'),
 ('Baby, Baby, Baby

In [36]:
pd.set_option('display.max_colwidth', -1)
data = pd.read_csv('decades-MusicBrainz.csv')
data['Clean_Title'] = data['Title'].replace(r'\([^)]*\)', '', regex=True).str.strip()

artist_dict = data.groupby('Title')['Artist'].apply(list).to_dict()

C:\Users\jojuh\AppData\Local\Temp\ipykernel_21236\2802442092.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Title,Artist,Genres,Lyric,Decade,Year,Popularity,Clean_Title
118,"You Don't Love Me (No, No, No)",Beyoncé,Pop; R&B; Black Music,"No no no\nYou don't love me\nAnd I know now\nNo no no\nYou don't love me\nYes, I know now\n'Cause you left me, baby\nAnd I got no place to go now\n\nNo no no\nI'll do anything you say boy\nNo no no\nI'll do anything you say boy\n'Cause if you ask me, baby\nI'll get on my knees and pray boy",1990,1994,205.5,You Don't Love Me


In [100]:
with open('wiki_songs.pickle', 'rb') as f:
    wiki = pickle.load(f)

df = pd.DataFrame(wiki, columns=['Title','URL','Year'])
df['Clean_Title'] = df['Title'].replace(r'\([^)]*\)', '', regex=True).str.strip()

df = df.merge(data, how='inner', on='Clean_Title')
df['URL_Year'] = df.apply(lambda x: (x['URL'], x['Year_x']), axis=1)

url_dict = df[['Clean_Title','URL','Year_x']].drop_duplicates()[['Clean_Title','URL_Year']].groupby('Clean_Title')['URL_Year'].apply(list).to_dict()

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'

In [107]:
url_dict = df.drop_duplicates(subset=['Clean_Title','URL','Year_x'])[['Clean_Title','URL_Year']].groupby('Clean_Title')['URL_Year'].apply(list).to_dict()

In [108]:
url_dict['Until I Die']

[('https://en.wikipedia.org/wiki/Until_I_Die', '2007')]

In [109]:
# import re
data[data['Clean_Title']=='Halo']

In [119]:
matches = []

with tqdm(total=len(artist_dict)) as pbar:
    for title, artist in artist_dict.items():
        # print(title)
        urls = url_dict.get(title)
        # print(urls)
        if urls:
            for u, y in urls:
                # print(u)
                page = requests.get(u)
                soup = BeautifulSoup(page.content, 'html.parser') # .find('div', {'id': 'mw-content-text'})
                for a in artist:
                    # print(u)
                    # print('Artist:', a)
                    # print('Trying panel...')
                    if soup.find_all('td', text=re.compile(a)):
                        matches.append((title, a, y))
                        # print('Pane:', title, a, y)
                        break
                    # print('Trying body...')
                    if soup.find_all('p', text=re.compile(a)):
                # if any(r['Artist'] in s for s in soup.strings):
                        matches.append((title, a, y))
                        # print('Body:', title, a, y)
                        break
            
            pbar.update(1)

with open('wiki_matches.pickle', 'wb') as handle:
    pickle.dump(matches, handle)

  5%|▌         | 1530/28941 [55:11<16:28:42,  2.16s/it] 


error: nothing to repeat at position 0

In [94]:
matches

[]

In [ ]:
songs_by_year

In [239]:
import pandas as pd
df = pd.DataFrame(songs_by_year, columns=['Year','Song','URL'])
len(df[df['URL']=='1950'])

831

In [253]:
df[df['URL']=='1950'].to_csv('delete_me.csv')